In [56]:
import pandas as pd
import numpy as np

In [69]:
def clean_excel_data(file_path, output_path=None):
    """
    Clean Excel data with multi-row headers starting from row 4.
    Handles merged cells and preserves comma-separated header names.
    
    Parameters:
    file_path (str): Path to the input Excel file
    output_path (str): Path for the output file (optional)
    
    Returns:
    pd.DataFrame: Cleaned dataframe
    """

    xls = pd.ExcelFile(file_path)
    last_sheet = xls.sheet_names[-1]
    print("✅ Loading last sheet:", last_sheet)

    df_raw = pd.read_excel(file_path, sheet_name=last_sheet, header=None, dtype=str)
    
    # Extract row 3 and row 4 (indices 2 and 3)
    header_row_1 = df_raw.iloc[2].fillna('')
    header_row_2 = df_raw.iloc[3].fillna('')
    
    # Combine headers: If row 4 has content, use it; otherwise use row 3
    # For merged cells, we'll concatenate with a space
    combined_headers = []
    
    for col_idx in range(len(header_row_1)):
        h1 = str(header_row_1.iloc[col_idx]).strip()
        h2 = str(header_row_2.iloc[col_idx]).strip()
        
        # Remove 'nan' strings that might appear
        h1 = '' if h1 == 'nan' else h1
        h2 = '' if h2 == 'nan' else h2
        
        # Combine headers intelligently
        if h1 and h2:
            # Both rows have content
            combined = f"{h1} - {h2}"
        elif h1:
            # Only row 3 has content
            combined = h1
        elif h2:
            # Only row 4 has content
            combined = h2
        else:
            # Neither has content, use column letter
            combined = f"Column_{col_idx}"
        
        combined_headers.append(combined)
    
    # Get the data starting from row 5 (index 4)
    df_clean = df_raw.iloc[4:].copy()
    
    # Set the combined headers
    df_clean.columns = combined_headers
    
    # Reset index
    df_clean.reset_index(drop=True, inplace=True)
    
    # Remove completely empty rows
    df_clean = df_clean.dropna(how='all')
    
    # Remove completely empty columns
    df_clean = df_clean.dropna(axis=1, how='all')
    
    # Clean up any remaining issues
    df_clean = df_clean.replace('nan', np.nan)
    
    # Save to output file if specified
    if output_path:
        df_clean.to_csv(output_path, index=False)
        print(f"Cleaned data saved to: {output_path}")
    
    return df_clean

In [70]:
def preview_headers(file_path, output_path):
    """
    Preview the first few rows including headers to verify structure.
    
    Parameters:
    file_path (str): Path to the Excel file
    num_rows (int): Number of data rows to preview
    """
    df_raw = pd.read_excel(file_path, sheet_name=-1, header=None)
    
    # Clean and show result
    df_clean = clean_excel_data(file_path, output_path)
    
    return df_clean

In [71]:
input_file = "data/SUT and IO By Divisions -En.xlsx"    
output_file = "cleaned_data/sut_io_cleaned_data.csv"

# Preview the data
df = preview_headers(input_file, output_file)


✅ Loading last sheet: 6.6
Cleaned data saved to: cleaned_data/sut_io_cleaned_data.csv
